In [3]:
import text_utilities as tu
import regex as rex
%load_ext autoreload
%autoreload 2

In [5]:

pc = tu.PosCorpus('../data/experiment/verbs')
pc.counted_corpus.keys()

dict_keys([3, 1, 2, 4, 5, 6, 7, 8])

In [78]:
class WordMapper:
    """Generates a Wordmap for a target token, comparing it to its POS-members in a dict sorted by syllables"""
    def __init__(self, target: str, tokenset: dict, clean=True, pattern='([^^1][0*1*]+[^$1])'):
        self.target = target
        self.tokenset = tokenset
        self.syllables = tu.count_syllables(target)
        self.maps = self.stack_maps(target, tokenset, self.syllables)
        self.pattern = pattern
        self.clean_maps = None
        if clean:
            self.filter_map_noise(self.maps, self.pattern)
            self.wordmap = self.sum_map_stack(self.clean_maps)
        else:
            self.wordmap = self.sum_map_stack(self.maps)


    def stack_maps(self, target, tokenset, syllables):
        l = len(target)
        cc1, cc2, cc3 = 0,0,0
        maps = []
        for k in tokenset:
            for v in tokenset[k]:
                pair = (v, self.target)
                case = tu.match_ends(v, target)
                shorter = min(pair, key=len)
                longer = max(pair, key=len)
                diff = len(longer) - len(shorter)


                if case.get("any"):
                    if diff:
                        if case.get("first") and syllables!=1:
                            cc1+=1
                            wm = tu.wordmap(longer=longer, shorter=shorter)
                            while len(wm) < l:
                                wm.append(0) # padding
                            maps.append(wm)

                        if case.get("last"):
                            wm = []
                            cc2+=1
                            wm = tu.wordmap(longer=longer, shorter=shorter, start=diff)
                            while len(wm) < l:
                                wm.insert(0, 0) # padding
                            maps.append(wm)


                    else:
                        cc3+=1
                        wm = tu.wordmap(longer=pair[0], shorter=pair[1])
                        maps.append(wm)
        print("Cases:", cc1, cc2, cc3)
        return maps


    def filter_map_noise(self, maps, pattern):
        """Convert maps to strings and delete any consecutive '1' not at the start or end of the map"""
        str_maps = ["".join([str(c) for c in m]) for m in maps]
        #for m in maps:
        #    mstr = ""
        #    for i in m:
        #        mstr += str(i)
        #    str_maps.append(mstr)

        recount_map = [rex.sub(pattern=pattern, repl=lambda m: len(m.group(1))*"0",string=sm) for sm in str_maps]  # substitute false positives
        #for cnt in str_maps:
        #    sr = rex.sub(pattern='([^^1][0*1*]+[^$1])', repl=lambda m: len(m.group(1))*"0",string=cnt)
        #    recount_map.append(sr)

        regexed_listed = [list(i) for i in recount_map]
        regexed_inted = [[int(c) for c in m] for m in regexed_listed]  # cast back to int

        #for i in regexed_listed:
        #    regexed_inted.append([int(j) for j in i])

        self.clean_maps = regexed_inted

    def sum_map_stack(self, maps):
        return [sum(x) for x in zip(*maps)]


In [81]:
wordmap = WordMapper("sehen", pc.counted_corpus, clean=False)

Cases: 10632 10685 385


In [82]:
wordmap.wordmap

[11667, 2440, 5956, 5560, 11199]

In [29]:
def raw_map(target, maps):
    """Returns a single map by stacking a list of maps"""
    new_maps = [x for x in maps if len(x)==len(target)]
    return [sum(x) for x in zip(*new_maps)]

'sehen'

In [29]:
class MapToken:
    def __init__(self, token, wm=list):
        self.wordmap = wm
        self.freqmap = map
        self.str = token
        self.str_list = list(map)

AttributeError: 'WordMap' object has no attribute 'ts'

[1092, 1059, 799, 1646, 7839, 24274]

000001 24413


In [11]:
import collections as cl
n_maps = cl.Counter(recount_map).most_common(50)
n_maps[:50]

[('000001', 16022),
 ('000011', 6215),
 ('000111', 972),
 ('100011', 473),
 ('100001', 343),
 ('100000', 105),
 ('001111', 89),
 ('110001', 71),
 ('110011', 57),
 ('110000', 33),
 ('011111', 23),
 ('100111', 7),
 ('111100', 1),
 ('111111', 1),
 ('101111', 1)]

In [44]:
def map_subword(target: str, map:str) -> str:
    """Returns a subword from a target string and a map. Yet to implement maps with 1 on both ends."""
    if map.startswith("1"):
        return target[:map.count("1")]+"##"
    elif map.endswith("1"):
        return "##"+target[-map.count("1"):]
print(map_subword("verstehen", "000000011"))

##en
